# Tutorial for srfpython

**Overview**
 + Intro : import and verify installation
 + I/ create a 1-D depth model  
 + II/ compute dispersion curves   
     + II.1/ use in a python program  
     + II.2/ use in command line   
 - III/ depth inversion  
 - III.1/ Program HerrMet  
 - III.2/ Application  
     + III.2.1/ Target data  
     + III.2.2/ Parameterization  
     + III.2.3/ Run inversion  
     + III.2.4/ Extract results

## Intro/ import and verify installation

In [ ]:
# -----------------------
# import all components of srfpython
# -----------------------
from srfpython import *

In [ ]:
# -----------------------
# make sure that the fortran codes have been compiled correctly
# -----------------------
try:
    check_herrmann_codes() 
    print "ok"
    
except Exception:
    print "compilation was not done or done on another system"

    # recompile
    from srfpython import recompile_src90
    recompile_src90(yes=False)

    # verify once more
    try:
        check_herrmann_codes()
        print 'sucess'
    except Exception:
        print 'Error : unexpected failure, make sure you have gfortran installed'



## I/ create a 1-D depth model

In [ ]:
# -----------------------
# create 1-D depth model using 4 arrays with same length
# -----------------------
ztop = [0.00, 0.25, 0.45, 0.65, 0.85, 1.05, 1.53, 1.80] #km, top layer depth, positive, increasing downward, 0 = surface
vp   = [1.85, 2.36, 2.63, 3.15, 3.71, 4.54, 5.48, 5.80] #km/s, P wave velocity in each layer
vs   = [0.86, 1.10, 1.24, 1.47, 1.73, 2.13, 3.13, 3.31] #km/s, S wave velocity in each layer
rh   = [2.47, 2.47, 2.47, 2.47, 2.47, 2.58, 2.58, 2.63] #g/cm3, Density in each layer

# create the depthmodel object, use a subclass that is to be intitiated with arrays
# see also depthmodel, depthmodel1D, depthmodel_from_mod96, ...
dm = depthmodel_from_arrays(ztop, vp, vs, rh)


# __str__ returns the file content at mod96 format, (see Herrmann CPS documentation)
print dm 

In [ ]:
# -----------------------
# write the depth model as a file at mod96 format (see Herrmann CPS documentation)
# -----------------------
dm.write96('dmtuto.mod96')

In [ ]:
# -----------------------
# display the depth model
# -----------------------
plt.figure(figsize=(2, 4))
dm.show(gca())
gca().set_title('figure 1 : dmtuto.mod96')
gca().grid(True, linestyle=":")
plt.legend();

## II/ compute dispersion curves 

### II.1/ use in a python program

In [ ]:
# -----------------------
# use one of the following functions from srfpython.Herrmann.Herrmann
# -----------------------
print help(dispersion)
# print help(dispersion_1)
# print help(dispersion_2)

In [ ]:
# -----------------------
# compute dispersion curves from the depthmodel above
# -----------------------

# define the dipsersion curves to compute
#          Wave(R/L) Type(C/U) Mode    Frequency array (Hz)             
Curves = [('R',      'U',      0,      freqspace(0.2, 3.5, 35, "log")), 
          ('R',      'U',      1,      freqspace(0.2, 3.5, 35, "log")), 
          ('R',      'C',      0,      freqspace(0.2, 3.5, 35, "log")), 
          ('R',      'C',      1,      freqspace(0.2, 3.5, 35, "log")), 
          ('L',      'U',      0,      freqspace(0.2, 3.5, 35, "log")), 
          ('L',      'U',      1,      freqspace(0.2, 3.5, 35, "log")), 
          ('L',      'C',      0,      freqspace(0.2, 3.5, 35, "log")), 
          ('L',      'C',      1,      freqspace(0.2, 3.5, 35, "log"))] 

# compute dispersion curves
with Timer('dispersion'):
    out = list(dispersion_2(ztop, vp, vs, rh, Curves)) # list is used to iterate over the generator

# display the results
ax = plt.gca()
for w, t, m, fs, us in out:
    ax.loglog(1. / fs, us, '+-', label = "%s%s%d" % (w, t, m))
ax.set_xlabel('period (s)')
ax.set_ylabel('velocity (km/s)')    
ax.grid(True, which = "major")
ax.grid(True, which = "minor")
logtick(ax, "xy")
ax.set_title('figure 2 : Herrmann.py demo')

plt.legend()
plt.show()

### II.2/ use in command line 

In [ ]:
# -----------------------
# compute dispersion curves, and save as surf96 file
# -----------------------

import os
os.system('rm -f dmtuto*.surf96')
%run -i ../../srfpython/bin/m96 --disp dmtuto.mod96 \
    -LC0 .1 10 30 plog \
    -RC1 .1 10 30 plog \
    -RU0 .1 10 30 plog \
    -save dmtuto.surf96

# if the srfpython/bin directory was added to the path as stated in README.md, simply use m96, s96, HerrMet, ...

In [ ]:
# -----------------------
# display output
# -----------------------

%run -i ../../srfpython/bin/s96 --show dmtuto.surf96 -inline

see also programs **s96** and **m96** that provide more manipulation tools  
for depth models and surface wave dispersion curves in command line

In [ ]:
%run -i ../../srfpython/bin/m96 --help

In [ ]:
%run -i ../../srfpython/bin/s96 --help

## III/ depth inversion

### III.1/ Program HerrMet

In [ ]:
# display the main help
%run -i ../../srfpython/bin/HerrMet -help

### III.2/ Application

we propose to invert the synthetic data generated in section I (dmtuto.surf96) and compare the inversion result to the actual model used to synthetize the data (i.e. dmtuto.mod96)

In [ ]:
# -----------------------
# assert section I was executed
# clear former temporary files from III
# -----------------------

import os, glob
assert os.path.exists("./dmtuto.surf96")

# clean up before running
os.system('rm -rf _HerrMet_*')

# list files in this directory
for _ in glob.iglob('./*'):
    print _, 

#### III.2.1/ Target data

In [ ]:
# display detailed help for one or more plugins
%run -i ../../srfpython/bin/HerrMet -help target

In [ ]:
# -----------------------
# set the dispersion curves to invert referred to as the target data
# the dispersion curve must be provided as surf96 format (see Herrmann's doc, CPS)
# I reproduce the file content into _Herrmann.target (also at surf96 format), 
# which can be modified manually before inversion 
# (e.g. resample, remove data points or modes, adjust uncertainties for weighting, ...)
# -----------------------

# the following command will (see HerrMet --help for command names): 
# - get the target dispersion curves from dmtuto.surf96 (--target), 
# - resample it between 0.25-1 Hz with 7 samples spaced logarithmically in period domain (-resamp),
# - adjust the uncertainties to 0.2 * velocity (i.e. constant uncertainty in logaritmic domain, --lunc), 
# - overwrite the target file if exists (-ot) 

%run -i ../../srfpython/bin/HerrMet \
    --target dmtuto.surf96 \
        -resamp 0.25 1.0 7 plog \
        -lunc 0.2 \
        -ot
            
# display the target file with s96 (-inline avoids the program to pause)
%run -i ../../srfpython/bin/s96 --show _HerrMet_dmtuto/_HerrMet.target -inline

# or directly with HerrMet (--display)
%run -i ../../srfpython/bin/HerrMet \
    --display \
        -inline

#### III.2.2/ Parameterization

In [ ]:
# display detailed help for one or more plugins
%run -i ../../srfpython/bin/HerrMet -help param

In [ ]:
# -----------------------
# set the parameter file, the parameters will be stored into _Herrmet.param
# use Herrmet to generate a template version
# and customize it manually before running the inversion
# -----------------------

# the following command will :
# - set the parameter file with 4 layers down to 3 km (--param)
# - the parameters values will be adjusted based on an existing depthmodel (here dmtuto.mod96, -basedon),
# - define the parameterization mode as depth, vs, vp/vs and density (-t mZVSPRRH)
#   some other modes are available
# - require vp, vs and density to be growing (i.e. add prior constraints to the offsets between layers, -growing),             
# - overwrite the paramfile if exists (-op) 
# - display (-display) without pausing (-inline), plot also the actual model (-m96)

%run -i ../../srfpython/bin/HerrMet \
        --param 4 3. \
            -basedon dmtuto.mod96 \
            -t  mZVSPRRH \
            -growing \
            -op \
        --display . \
            -m96 dmtuto.mod96 \
            -inline

> red dashed curves = prior boundaries (locked for now)   
> green duspersion curves = target data  
> purple model = actual model used to generate the synthetic data and to build the parameterization    

> * Note that at this step, the boundaries for each parameter (red dashed curves)   
> are the same (because VINF=VSUP in _HerrMet.param) : i.e. all parameters are locked  
> one need to adjust the VINF, VSUP boundaries for all parameters to invert  
> you may do it manually (edit _HerrMet.param),   
> here I do it with programming tools for tutorial  


> * Note also that VP is not a parameter in this example, (since we use VS and VP/VS)  
> so the boundaries displayed on the VP axis are inferred from the VS and VP/VS ones
>


In [ ]:
# -----------------------
# customize the parameterization file using programming tools (for tutorial)
# You may do it manually simply by editing _Herrmet.param
# -----------------------

#load the parameter file, find lines related to top depth and to VS
A = AsciiFile('_HerrMet.param')

IZ  = np.asarray(["Z"  in _ for _ in A['KEY']], bool) #lines corresponding to Z parameters
IVS = np.asarray(["VS" in _ for _ in A['KEY']], bool) #lines corresponding to VS parameters
IPR2 = np.asarray(["PR2" in _ for _ in A['KEY']], bool) #line corresponding to VP/VS in the third layer

In [ ]:
# change parameter boundaries (decrease VINF and increase VSUP), overwrite _HerrMet.param
A['VINF'][IVS] = [0.55, 0.78, 1.53, 1.65]
A['VSUP'][IVS] = [2.22, 3.15, 4.00, 4.00]
A['VINF'][IZ]  = [-.31, -1.5, -3.1]
A['VSUP'][IZ]  = [-.11, -1.1, -2.0]
A['VINF'][IPR2] = A['VSUP'][IPR2] = 1.752
print A

# overwrite the parameterization file
A.write('_HerrMet.param')

In [ ]:
# -----------------------
# send the custom version of the parameterization file to the temporary directory
# -----------------------
%run -i ../../srfpython/bin/HerrMet -help send

In [ ]:
%run -i ../../srfpython/bin/HerrMet --send -op

In [ ]:
# -----------------------
# display the new version of the parameterization file
# -----------------------

#note that the boundaries now allow VS and Zop to vary between the red dashed lines
%run -i ../../srfpython/bin/HerrMet \
    --display \
        -inline \
        -m96 ./dmtuto.mod96

#### III.2.3/ Run inversion

In [ ]:
%run ../../srfpython/bin/HerrMet -help run

In [ ]:
# -----------------------
# run the inversion, will load the parameterization and target data,
# and generate markov chains to sample the posterior pdf (in parallel)
# the models generated by the chains will be stored in a sqlite database (_HerrMet.run)
# -----------------------

# the following command will :
# - run the inversion with 12 markov chains in restart mode (overwrites _HerrMet.run if exists)
# - each chain will be asked to keep 100 models
# - use 4 virtual threads (-w) affected to the first 4 physical threads (-taskset)
%run -i ../../srfpython/bin/HerrMet \
    -w 4 \
    -taskset "0-3" \
    -verbose 0 \
    --run \
        -nchain 12 \
        -nkeep 100 \
print "DONE"
        
# notations :
# kept : the number of models kept by the markov chain over the number of tests
# fail : some models can lead to failure of the forward algo (CPS), 
#        we consider them as models with no image in the dataspace
#        the penalty is adjusted to force the chains to move away from these dead ends
# AK   : Average keep ratio : the number of models kept / the number of tests
#        by default, the proposal pdf is adjusted to maintain this value around .25
# MP   : Master proposal : a coefficient to adjust the proposal distance according to AK
# AS   : Average speed : the average number of models tested per second and per chain
# LI   : Final log likelyhood = the quality of the last model found (not necessarily the best)

In [ ]:
# -----------------------
# see the convergence of the chains, delete chains or bad models
# -----------------------
%run -i ../../srfpython/bin/HerrMet -help manage

In [ ]:
%run -i ../../srfpython/bin/HerrMet -verbose 0 --manage -delbad -100. -plot -inline

In [ ]:
# -----------------------
# display the results, by selecting models in the sqlite database (_HerrMet.run)
# -----------------------

# the following command will :
# - display the best 200 models found and their image in the dataspace (-plot)
# - recompute the dispersion curves with higher resolution (-overdisp)
# - compute the median and std of the full population of models (-pdf)
# - add the model used to synthetize the data (dmtuto.mod96) for comparison (-m96)
%run -i ../../srfpython/bin/HerrMet -help display
%run -i ../../srfpython/bin/HerrMet \
        -verbose 0 \
        --display \
            -plot best 200 0. 1 \
            -overdisp \
            -pdf \
            -inline \
            -m96 dmtuto.mod96


> black doted curves = prior boundaries  
> red dispersion curves = target data  

> colored models = 200 best models sorted by increaseing likelyhood (see colorbar)   
> colored dispersion curves = corresponding data, recomputed with higher resolution  

> dark lines = median (thick), 16% 84% percentiles (thin) computed from the full population of models = solution  
> purple model = actual model used to generate the synthetic data = expected solution  


#### III.2.4/ Extract results

In [ ]:
# -----------------------
# HerrMet can create figures, 
# however you probably need to get the results at numerical format for further analysis
# use option --extract to compute and save the posterior pdf purcentiles
# -----------------------

# the following command will:
# -compute the median and std of the best 1000 models found, 
# -save it as mod96 files named 
#     _HerrMet.p0.16.mod96,_HerrMet.p0.50.mod96 and _HerrMet.p0.84.mod96
%run -i ../../srfpython/bin/HerrMet -help extract
%run -i ../../srfpython/bin/HerrMet \
    -verbose 0 \
    --extract \
        -pdf best 1000 0.0 1 

In [ ]:
#display models from --extract using m96
%run -i ../../srfpython/bin/m96 --show _HerrMet_*/_HerrMet.p*.mod96 dmtuto.mod96 -inline

In [ ]:
# clear temporary files
import glob, os
os.system('rm -rf dmtuto.*96 _HerrMet*')
for _ in glob.iglob('./*'):
    print _,